In [1]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import pickle

from torchmetrics.regression import SymmetricMeanAbsolutePercentageError
import torch
import os
from imblearn.over_sampling import SMOTE
import numpy as np

In [2]:
def metric(y_pred, y_test):
    ad_smape = SymmetricMeanAbsolutePercentageError()
    smape_adjust = torch.mean(torch.stack([ad_smape(y_pred.detach().cpu()[i], y_test.detach().cpu()[i]) * 0.5 for i in range(len(y_test))]))
    return smape_adjust

# KNN Classifier

In [3]:
data_path = '/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/'
meta_df = pd.read_csv(os.path.join(data_path,'meta_data_image_text_nofilter.csv'), index_col='item_number')
meta_df = meta_df.drop(['sales_std'], axis=1)
# meta_df = pd.read_csv(os.path.join(data_path,'240109_all_meta_sales_total.csv'), index_col='item_number')
# meta_df = meta_df.drop(['sales_total','sales_std'], axis=1)
meta_df

,sales_mean,fabric_WOVEN,fabric_DENIM,fabric_JERSEY,fabric_SWEATER,fabric_특종,main_color_MULTI_COLOR,main_color_BLUE,main_color_WHITE,main_color_PURPLE,...,main_color_RED,main_color_BROWN,main_color_GREY,main_color_YELLOW,main_color_ORANGE,category_OP,category_DP,category_HP,category_PT,category_WP
item_number,,,,,,,,,,,,,,,,,,,,,
JPOP123A,4.833333,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123B,5.833333,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123C,6.166667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
JPOP123D,1.083333,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
JPOP221A,0.500000,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MWPT71A1,65.666667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
MWPT71A2,49.666667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
MWPT71A3,13.166667,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
qcut_df = pd.read_csv("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/qcut_value_df_bin10.csv", index_col='품번')
qcut_df.loc[qcut_df['qcut_label'] != 0, 'qcut_label'] = 1
# n_classes=3
# qcut_df['qcut_label'] = pd.cut(qcut_df['sales_mean'], bins=n_classes, labels=range(0,n_classes))
qcut_label_mean = qcut_df.groupby('qcut_label')['sales_mean'].mean().values
qcut_label_median = qcut_df.groupby('qcut_label')['sales_mean'].median().values

# df = meta_df.drop('sales_mean', axis=1)
df = meta_df.merge(qcut_df['qcut_label'], left_index=True, right_index=True)

In [5]:
test_list = pickle.load(open("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/12salesweek_test_item_number296.pkl", 'rb')).drop("MTPT6102")[:]
train_list = df.index[~df.index.isin(test_list)]

train_df = df.loc[train_list]
test_df = df.loc[test_list]

X_train = train_df.drop(['qcut_label', 'sales_mean'], axis=1).values
y_train = train_df['qcut_label'].values
X_test = test_df.drop(['qcut_label', 'sales_mean'], axis=1).values
y_test = test_df['sales_mean'].values

In [34]:
# SMOTE
print(np.unique(y_train, return_counts=True))
sm = SMOTE(sampling_strategy={i:1300 for i in range(2)}, k_neighbors=10)
# PCA
# from sklearn.decomposition import PCA 
# pca = PCA(n_components=2)
# pca.fit(X_train) 
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

(array([0, 1]), array([1267,  174]))
(array([0, 1]), array([1300, 1300]))


In [71]:
# train dataset
# model = KNeighborsClassifier(n_neighbors=10)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_train)

# mean_pred = torch.FloatTensor(qcut_label_mean[y_pred])
# mean_gt = torch.FloatTensor(qcut_label_mean[y_train])

# median_pred = torch.FloatTensor(qcut_label_median[y_pred])
# median_gt = torch.FloatTensor(qcut_label_median[y_train])
# metric(mean_pred, mean_gt), metric(median_pred, median_gt)

# test dataset
model = KNeighborsClassifier(n_neighbors=7)
# model.fit(X_train, y_train)
X_train_os, y_train_os = sm.fit_resample(X_train, y_train)
print(np.unique(y_train_os, return_counts=True))

model.fit(X_train_os, y_train_os)
y_pred = model.predict(X_test)

mean_pred = torch.FloatTensor(qcut_label_mean[y_pred])
mean_gt = torch.FloatTensor(y_test)

median_pred = torch.FloatTensor(qcut_label_median[y_pred])
median_gt = torch.FloatTensor(y_test)

metric(mean_pred, mean_gt), metric(median_pred, median_gt)

(array([0, 1]), array([1300, 1300]))


(tensor(0.4698), tensor(0.3723))

# metadata groupby

In [8]:
data_path = '/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/'
meta_df = pd.read_csv(os.path.join(data_path,'meta_data_image_text_nofilter.csv'), index_col='item_number')
meta_df = meta_df.drop(['sales_std'], axis=1)

df = pd.DataFrame()

df['fabric'] = meta_df.iloc[:,1:6].idxmax(axis=1).apply(lambda x:x.split('_')[-1])
df['color'] = meta_df.iloc[:,6:19].idxmax(axis=1).apply(lambda x:x.split('_')[-1])
df['category'] = meta_df.iloc[:,19:].idxmax(axis=1).apply(lambda x:x.split('_')[-1])
df['sales_mean'] = meta_df.iloc[:,0]

test_list = pickle.load(open("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/12salesweek_test_item_number296.pkl", 'rb')).drop("MTPT6102")[:]
train_list = df.index[~df.index.isin(test_list)]

train_df = df.loc[train_list]
test_df = df.loc[test_list]

qcut_df = pd.read_csv("/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/qcut_value_df_bin10.csv", index_col='품번')
qcut_label_mean = qcut_df.groupby('qcut_label')['sales_mean'].mean().values
qcut_label_median = qcut_df.groupby('qcut_label')['sales_mean'].median().values

In [9]:
use_cols = ['fabric', 'color', 'category']
group_df = train_df.groupby(use_cols)['sales_mean'].mean()
group_df.name = 'mean_pred'
temp_df = test_df.merge(group_df, on=use_cols)

group_df = train_df.groupby(use_cols)['sales_mean'].median()
group_df.name = 'median_pred'
temp_df = temp_df.merge(group_df, on=use_cols)

mean_pred = torch.FloatTensor(temp_df['mean_pred'].values)
median_pred = torch.FloatTensor(temp_df['median_pred'].values)
y_test = torch.FloatTensor(temp_df['sales_mean'].values)
metric(mean_pred, y_test), metric(median_pred, y_test)

(tensor(0.3169), tensor(0.3170))

# Only 0 bin

In [10]:
# only 0 bin
mean_pred = torch.FloatTensor([qcut_label_mean[0]]*y_test.shape[0])
median_pred = torch.FloatTensor([qcut_label_median[0]]*y_test.shape[0])
metric(mean_pred, y_test), metric(median_pred, y_test)

(tensor(0.4892), tensor(0.4523))

# Mean of train dataset

In [11]:
mean_pred = torch.FloatTensor([train_df['sales_mean'].mean()]*y_test.shape[0])
median_pred = y_pred = torch.FloatTensor([train_df['sales_mean'].median()]*y_test.shape[0])
metric(mean_pred, y_test), metric(median_pred, y_test)

(tensor(0.5736), tensor(0.4610))